# DIMPLE3 Simulation

DIMPLE3 is the Euler's method extension of the ordinary DIMPLE system which aims to solve three major problems with DIMPLE:

1. Exponential space requirment in coupling precision
2. Underutilization of quadratic space in number of spins
3. Suboptimal convergence from deterministic dynamics

To this end, we propose an architecture which extends the buffer-chain DIMPLE hardware with three corresponding improvements:

1. Replace coupling cell rows with single spin counters
2. Allow for comparison between these counters via a barrel shifter
3. Add back to spin counters with appropriate coupling weight and noise

The following is an event driven simulation of the proposed architecture with which to benchmark any FPGA implementations.

Presents with the following advantages:

- Euler simulation requires O(N) clock cycles for N spins
- Lin-Log Space allows for simple Trotterization via replication
- Multiple instances of circuit only require one copy of memory to be shared

Has the following disadvantages:

- Comparison is expensive especially through a barrel-shifter, requiring a very high constant multiplier on the theoretical big-O lin-log term or approximation heuristics.

In [ ]:
import numpy as np

num_copies = 3
num_spins = 5
max_state = 1 << 10

def compare_adjacent_columns(arr):
    # Get number of columns
    n_cols = arr.shape[1]

    # Initialize results array
    comparisons = np.zeros((arr.shape[0], n_cols), dtype=bool)

    # Compare each column with next column
    for i in range(n_cols):
        next_col = (i + 1) % n_cols  # Wrap around to 0 for last column
        comparisons[:, i] = arr[:, i] > arr[:, next_col]

    return comparisons

# Replicated state registers for multiple lin-log circuits
state_array = np.zeros((num_copies,num_spins))

# For memory efficiency, we note that this will only have
# single num_spins x num_spins or num_spins array in memory.
coupling_array = np.ones((num_copies,num_spins,num_spins))
drift_array = np.zeros((num_copies,num_spins))

trotterization_strength = 1

# For now this is empty, but you might want to simulate an LFSR, etc
noise_generator = lambda x : np.zeros((num_copies,num_spins))

loops = 100

for _ in range(loops):

    # We count up through the indices for each iterloop
    # to go completely around the barrel shifter

    # NOTE: one step in Euler's method for this machine takes O(#spins)
    for i in range(spins):

        # This is the action of the barrel shifter
        rotated = np.roll(state_array, i, axis=1)

        # Simpler comparison logic can be used
        # but this is the primary function that
        # we want to emulate in the machine.
        comparison = state_array > rotated

        # This selects the appropriate instance of memory tensor
        circular_buffer = np.diagonal(tensor, offset=i-(spins//2), axis1=1, axis2=2)

        # Adds correct members of circular buffer and adds a noise term
        state_array += np.where(comparison,circular_buffer,0) + noise_generator()

    # Add the relevant drift terms
    state_array += drift_array

    # Transmit Trotterization data
    state_array += np.where(compare_adjacent_columns(state_array),trotterization_strength,0)

    # Handle state overflow
    state_array = state_array % max_state